In [2]:
%%capture
!pip install --force-reinstall --no-cache-dir tenacity --user
!pip install "ibm-watsonx-ai==1.0.4" --user
!pip install "ibm-watson-machine-learning==1.0.357" --user
!pip install "langchain-ibm==0.1.7" --user
!pip install "langchain-community==0.2.1" --user
!pip install "langchain-experimental==0.0.59" --user
!pip install "langchainhub==0.1.17" --user
!pip install "langchain==0.2.1" --user
!pip install "pypdf==4.2.0" --user
!pip install "chromadb == 0.4.24" --user

In [1]:
# Suppress warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# Import required libraries
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import WatsonxEmbeddings

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_core.prompts import PromptTemplate
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain import hub
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.documents import Document
import os
import re
from datetime import datetime
from typing import Dict, List, Optional, Tuple
from enum import Enum

class MedicalDomain(Enum):
    DIABETES = "diabetes"
    CANCER = "cancer"
    MENTAL_HEALTH = "mental_health"
    CARDIAC = "cardiac"
    GENERAL = "general"

class BaseMedicalAgent:
    """Base class for all medical agents"""
    
    def __init__(self, domain: MedicalDomain, domain_name: str):
        self.domain = domain
        self.domain_name = domain_name
        self.setup_llm()
        self.setup_embeddings()
        self.knowledge_base = None
        self.conversation_chain = None
        
    def setup_llm(self):
        """Setup the Watson LLM with optimized parameters for medical discussions"""
        model_id = 'mistralai/mistral-large'
        
        parameters = {
            GenParams.MAX_NEW_TOKENS: 512,
            GenParams.TEMPERATURE: 0.1,
            GenParams.TOP_P: 0.9,
            GenParams.REPETITION_PENALTY: 1.1
        }
        
        credentials = {
            "url": "https://us-south.ml.cloud.ibm.com"
        }
        
        project_id = "skills-network"
        
        model = ModelInference(
            model_id=model_id,
            params=parameters,
            credentials=credentials,
            project_id=project_id
        )
        
        self.llm = WatsonxLLM(model=model)
        
    def setup_embeddings(self):
        """Setup Watson embeddings for document retrieval"""
        embed_params = {
            EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
            EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
        }
        
        self.embeddings = WatsonxEmbeddings(
            model_id="ibm/slate-125m-english-rtrvr",
            url="https://us-south.ml.cloud.ibm.com",
            project_id="skills-network",
            params=embed_params,
        )
    
    def load_knowledge_base(self, pdf_paths: List[str]) -> Tuple[int, int]:
        """Load knowledge base from PDF files"""
        all_documents = []
        successful_loads = 0
        
        for i, pdf_path in enumerate(pdf_paths):
            try:
                print(f"🔄 Loading {self.domain_name} textbook {i+1}: {pdf_path}")
                
                if not os.path.exists(pdf_path):
                    print(f"⚠️ File not found: {pdf_path}")
                    continue
                    
                if not os.access(pdf_path, os.R_OK):
                    print(f"⚠️ Cannot read file: {pdf_path}")
                    continue
                
                loader = PyPDFLoader(pdf_path)
                documents = loader.load()
                
                if not documents:
                    print(f"⚠️ No documents loaded from {pdf_path}")
                    continue
                    
                print(f"Loaded {len(documents)} pages from {self.domain_name} textbook {i+1}")
                
                for doc in documents:
                    doc.metadata.update({
                        'source_textbook': f'{self.domain.value}_textbook_{i+1}',
                        'source_path': pdf_path,
                        'filename': os.path.basename(pdf_path),
                        'domain': self.domain.value
                    })
                
                all_documents.extend(documents)
                successful_loads += 1
                print(f"✅ Successfully added {self.domain_name} textbook {i+1}")
                
            except Exception as e:
                print(f"❌ Error loading {pdf_path}: {e}")
                continue
        
        if all_documents:
            print(f"📖 Total {self.domain_name} documents: {len(all_documents)} pages")
            
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200,
                separators=["\n\n", "\n", ". ", " ", ""],
                length_function=len,
            )
            
            chunks = text_splitter.split_documents(all_documents)
            print(f"Created {len(chunks)} text chunks for {self.domain_name}")
            
            try:
                self.knowledge_base = Chroma.from_documents(
                    chunks, 
                    self.embeddings,
                    collection_name=f"{self.domain.value}_knowledge"
                )
                print(f"✅ {self.domain_name} knowledge base created successfully!")
                return len(chunks), successful_loads
                
            except Exception as embed_error:
                print(f"Embedding error for {self.domain_name}: {embed_error}")
                return self._create_knowledge_base_in_batches(chunks)
        else:
            print(f"❌ No {self.domain_name} textbooks loaded. Creating demo version...")
            return self.create_demo_knowledge_base()
    
    def _create_knowledge_base_in_batches(self, chunks: List) -> Tuple[int, int]:
        """Create knowledge base in smaller batches if embedding fails"""
        print(f"Trying {self.domain_name} knowledge base creation with smaller batches...")
        batch_size = 50
        processed_chunks = []
        
        for i in range(0, len(chunks), batch_size):
            batch = chunks[i:i + batch_size]
            try:
                if i == 0:
                    self.knowledge_base = Chroma.from_documents(
                        batch, 
                        self.embeddings,
                        collection_name=f"{self.domain.value}_knowledge"
                    )
                else:
                    self.knowledge_base.add_documents(batch)
                processed_chunks.extend(batch)
                print(f"Processed {self.domain_name} batch {i//batch_size + 1}/{(len(chunks)-1)//batch_size + 1}")
            except Exception as batch_error:
                print(f"{self.domain_name} batch {i//batch_size + 1} failed: {batch_error}")
                continue
        
        return len(processed_chunks), 1 if processed_chunks else 0
    
    def create_demo_knowledge_base(self) -> Tuple[int, int]:
        """Create demo knowledge base - to be overridden by subclasses"""
        raise NotImplementedError("Subclasses must implement create_demo_knowledge_base")
    
    def create_medical_prompt(self) -> PromptTemplate:
        """Create specialized prompt template - to be overridden by subclasses"""
        raise NotImplementedError("Subclasses must implement create_medical_prompt")
    
    def setup_conversation_chain(self):
        """Setup the conversational retrieval chain"""
        if self.knowledge_base is None:
            raise ValueError(f"{self.domain_name} knowledge base not loaded.")
            
        retriever = self.knowledge_base.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 5}
        )
        
        prompt = self.create_medical_prompt()
        
        # Create temporary memory for this agent (will be managed by orchestrator)
        temp_memory = ConversationBufferWindowMemory(
            k=10,
            memory_key="chat_history",
            return_messages=True,
            output_key="answer"
        )
        
        self.conversation_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=retriever,
            memory=temp_memory,
            return_source_documents=True,
            combine_docs_chain_kwargs={"prompt": prompt},
            verbose=False
        )
    
    def ask_question(self, question: str, chat_history: List = None) -> Dict:
        """Ask a question and get response"""
        if self.conversation_chain is None:
            raise ValueError(f"{self.domain_name} conversation chain not setup.")
        
        # Update memory with chat history if provided
        if chat_history:
            self.conversation_chain.memory.chat_memory.messages = chat_history
        
        formatted_question = f"[{self.domain_name} Query - {datetime.now().strftime('%Y-%m-%d %H:%M')}] {question}"
        
        try:
            result = self.conversation_chain({"question": formatted_question})
            
            return {
                "answer": result["answer"],
                "source_documents": result.get("source_documents", []),
                "confidence": "High" if len(result.get("source_documents", [])) >= 3 else "Medium",
                "domain": self.domain.value,
                "agent": self.domain_name
            }
            
        except Exception as e:
            return {
                "answer": f"I apologize, but I encountered an error processing your {self.domain_name} query: {str(e)}. Please try rephrasing your question.",
                "source_documents": [],
                "confidence": "Low",
                "domain": self.domain.value,
                "agent": self.domain_name
            }

class DiabeticAgent(BaseMedicalAgent):
    """Specialized agent for diabetes-related queries"""
    
    def __init__(self):
        super().__init__(MedicalDomain.DIABETES, "Diabetes")
    
    def create_demo_knowledge_base(self) -> Tuple[int, int]:
        """Create demo diabetes knowledge base"""
        print("Creating demo diabetes knowledge base...")
        
        diabetes_content = [
            """
            Type 2 Diabetes Mellitus Diagnostic Criteria:
            
            The American Diabetes Association (ADA) diagnostic criteria for Type 2 diabetes mellitus include:
            
            1. Fasting Plasma Glucose (FPG) ≥ 126 mg/dL (7.0 mmol/L)
            2. 2-hour Plasma Glucose ≥ 200 mg/dL (11.1 mmol/L) during OGTT
            3. HbA1c ≥ 6.5% (48 mmol/mol)
            4. Random plasma glucose ≥ 200 mg/dL (11.1 mmol/L) with classic symptoms
            
            Each test should be confirmed on a separate day unless the patient has classic symptoms of hyperglycemia.
            """,
            """
            Diabetic Ketoacidosis (DKA) Pathophysiology:
            
            DKA is a serious complication of diabetes characterized by:
            
            1. Insulin deficiency leading to increased lipolysis
            2. Elevated ketone production (β-hydroxybutyrate and acetoacetate)
            3. Metabolic acidosis with anion gap
            4. Severe dehydration and electrolyte imbalances
            
            Clinical presentation includes polyuria, polydipsia, abdominal pain, vomiting, and altered mental status.
            
            Treatment involves IV insulin, fluid resuscitation, and electrolyte replacement.
            """,
            """
            Metformin Therapy Guidelines:
            
            Metformin is the first-line therapy for Type 2 diabetes:
            
            1. Mechanism: Reduces hepatic glucose production, improves insulin sensitivity
            2. Starting dose: 500-850 mg once or twice daily with meals
            3. Maximum dose: 2550 mg daily (divided doses)
            
            Contraindications:
            - eGFR <30 mL/min/1.73m²
            - Acute kidney injury
            - Severe liver disease
            - Heart failure requiring hospitalization
            """
        ]
        
        documents = []
        for i, content in enumerate(diabetes_content):
            doc = Document(
                page_content=content,
                metadata={
                    'source': f'diabetes_demo_section_{i+1}',
                    'topic': ['Diagnosis', 'DKA', 'Metformin'][i],
                    'domain': 'diabetes'
                }
            )
            documents.append(doc)
        
        self.knowledge_base = Chroma.from_documents(
            documents, 
            self.embeddings,
            collection_name="diabetes_demo"
        )
        
        print(f"Demo diabetes knowledge base created with {len(documents)} documents!")
        return len(documents), 1
    
    def create_medical_prompt(self) -> PromptTemplate:
        """Create diabetes-specific prompt template"""
        template = """You are DiabetesGPT, an advanced AI medical assistant specializing in diabetes care. You assist with diabetes diagnosis, treatment protocols, pathophysiology, and patient management.

MEDICAL GUIDELINES:
1. Base responses on provided diabetes literature
2. Provide evidence-based, clinically accurate information
3. Include diagnostic criteria, treatment guidelines, and clinical considerations
4. Use appropriate medical terminology
5. Mention when specialist consultation may be needed

DIABETES KNOWLEDGE BASE:
{context}

CONVERSATION HISTORY:
{chat_history}

DIABETES QUERY: {question}

CLINICAL RESPONSE:
Provide a comprehensive, evidence-based response addressing the diabetes query. Include relevant clinical context, treatment considerations, and follow-up recommendations.

Response:"""

        return PromptTemplate(
            template=template,
            input_variables=["context", "chat_history", "question"]
        )

class CancerAgent(BaseMedicalAgent):
    """Specialized agent for cancer-related queries"""
    
    def __init__(self):
        super().__init__(MedicalDomain.CANCER, "Cancer/Oncology")
    
    def create_demo_knowledge_base(self) -> Tuple[int, int]:
        """Create demo cancer knowledge base"""
        print("Creating demo cancer knowledge base...")
        
        cancer_content = [
            """
            Breast Cancer Staging and Treatment:
            
            TNM Staging System for Breast Cancer:
            - T (Tumor size): T1 (<2cm), T2 (2-5cm), T3 (>5cm), T4 (chest wall/skin involvement)
            - N (Lymph nodes): N0 (none), N1 (1-3 axillary), N2 (4-9 axillary), N3 (>9 or other regions)
            - M (Metastasis): M0 (none), M1 (distant metastasis)
            
            Treatment approaches:
            - Surgery: Lumpectomy vs. mastectomy
            - Adjuvant therapy: Chemotherapy, radiation, hormonal therapy
            - Targeted therapy: HER2-positive tumors (trastuzumab)
            - Immunotherapy: Triple-negative breast cancer considerations
            """,
            """
            Lung Cancer Classification and Management:
            
            Two main types:
            1. Non-Small Cell Lung Cancer (NSCLC) - 85% of cases
               - Adenocarcinoma (most common)
               - Squamous cell carcinoma
               - Large cell carcinoma
            
            2. Small Cell Lung Cancer (SCLC) - 15% of cases
               - Limited stage vs. extensive stage
               - More aggressive, better initial response to chemotherapy
            
            Staging: Stage I-IV based on tumor size, lymph node involvement, and metastasis
            Treatment: Surgery, chemotherapy, radiation, targeted therapy, immunotherapy
            """,
            """
            Chemotherapy Principles and Side Effects:
            
            Mechanisms of action:
            - Alkylating agents: DNA cross-linking
            - Antimetabolites: Interfere with DNA synthesis
            - Topoisomerase inhibitors: Prevent DNA unwinding
            - Mitotic inhibitors: Disrupt cell division
            
            Common side effects:
            - Myelosuppression (neutropenia, anemia, thrombocytopenia)
            - Gastrointestinal toxicity (nausea, vomiting, mucositis)
            - Alopecia
            - Peripheral neuropathy
            - Cardiotoxicity (doxorubicin)
            - Pulmonary toxicity (bleomycin)
            """
        ]
        
        documents = []
        for i, content in enumerate(cancer_content):
            doc = Document(
                page_content=content,
                metadata={
                    'source': f'cancer_demo_section_{i+1}',
                    'topic': ['Breast Cancer', 'Lung Cancer', 'Chemotherapy'][i],
                    'domain': 'cancer'
                }
            )
            documents.append(doc)
        
        self.knowledge_base = Chroma.from_documents(
            documents, 
            self.embeddings,
            collection_name="cancer_demo"
        )
        
        print(f"Demo cancer knowledge base created with {len(documents)} documents!")
        return len(documents), 1
    
    def create_medical_prompt(self) -> PromptTemplate:
        """Create cancer-specific prompt template"""
        template = """You are OncologyGPT, an advanced AI medical assistant specializing in cancer care and oncology. You assist with cancer diagnosis, staging, treatment protocols, and patient management.

ONCOLOGY GUIDELINES:
1. Base responses on provided oncology literature
2. Provide evidence-based cancer treatment information
3. Include staging criteria, treatment protocols, and prognosis considerations
4. Use appropriate oncological terminology
5. Emphasize multidisciplinary care approach
6. Mention when specialist consultation is essential

ONCOLOGY KNOWLEDGE BASE:
{context}

CONVERSATION HISTORY:
{chat_history}

ONCOLOGY QUERY: {question}

CLINICAL RESPONSE:
Provide a comprehensive, evidence-based response addressing the cancer/oncology query. Include relevant staging information, treatment options, side effect management, and multidisciplinary care considerations.

Response:"""

        return PromptTemplate(
            template=template,
            input_variables=["context", "chat_history", "question"]
        )

class MentalHealthAgent(BaseMedicalAgent):
    """Specialized agent for mental health queries"""
    
    def __init__(self):
        super().__init__(MedicalDomain.MENTAL_HEALTH, "Mental Health/Psychiatry")
    
    def create_demo_knowledge_base(self) -> Tuple[int, int]:
        """Create demo mental health knowledge base"""
        print("Creating demo mental health knowledge base...")
        
        mental_health_content = [
            """
            Major Depressive Disorder (MDD) Diagnosis and Treatment:
            
            DSM-5 Criteria for Major Depressive Episode:
            At least 5 symptoms for ≥2 weeks, including depressed mood or anhedonia:
            1. Depressed mood most of the day
            2. Anhedonia (loss of interest/pleasure)
            3. Significant weight change or appetite disturbance
            4. Sleep disturbance (insomnia or hypersomnia)
            5. Psychomotor agitation or retardation
            6. Fatigue or loss of energy
            7. Feelings of worthlessness or excessive guilt
            8. Diminished concentration or indecisiveness
            9. Recurrent thoughts of death or suicidal ideation
            
            Treatment approaches:
            - First-line: SSRIs (sertraline, escitalopram, fluoxetine)
            - Psychotherapy: CBT, IPT, behavioral activation
            - Combination therapy for moderate-severe depression
            """,
            """
            Anxiety Disorders Classification and Management:
            
            Types of Anxiety Disorders:
            1. Generalized Anxiety Disorder (GAD)
            2. Panic Disorder
            3. Social Anxiety Disorder
            4. Specific Phobias
            5. Agoraphobia
            
            GAD Diagnostic Criteria:
            - Excessive anxiety/worry for ≥6 months
            - Difficult to control worry
            - Associated with ≥3 symptoms: restlessness, fatigue, concentration problems, irritability, muscle tension, sleep disturbance
            
            Treatment:
            - First-line: SSRIs, SNRIs
            - Psychotherapy: CBT, exposure therapy
            - Short-term: Benzodiazepines (avoid long-term use)
            """,
            """
            Bipolar Disorder Management:
            
            Bipolar I vs. Bipolar II:
            - Bipolar I: At least one manic episode
            - Bipolar II: Hypomanic episodes with major depressive episodes
            
            Manic Episode Criteria:
            - Elevated, expansive, or irritable mood for ≥1 week
            - ≥3 symptoms: Grandiosity, decreased sleep, talkativeness, flight of ideas, distractibility, increased goal-directed activity, risky behavior
            
            Treatment:
            - Mood stabilizers: Lithium, valproate, lamotrigine
            - Atypical antipsychotics: Quetiapine, olanzapine, aripiprazole
            - Psychotherapy: CBT, family therapy, psychoeducation
            - Avoid antidepressant monotherapy (risk of mania induction)
            """
        ]
        
        documents = []
        for i, content in enumerate(mental_health_content):
            doc = Document(
                page_content=content,
                metadata={
                    'source': f'mental_health_demo_section_{i+1}',
                    'topic': ['Depression', 'Anxiety', 'Bipolar'][i],
                    'domain': 'mental_health'
                }
            )
            documents.append(doc)
        
        self.knowledge_base = Chroma.from_documents(
            documents, 
            self.embeddings,
            collection_name="mental_health_demo"
        )
        
        print(f"Demo mental health knowledge base created with {len(documents)} documents!")
        return len(documents), 1
    
    def create_medical_prompt(self) -> PromptTemplate:
        """Create mental health-specific prompt template"""
        template = """You are PsychiatryGPT, an advanced AI medical assistant specializing in mental health and psychiatric care. You assist with psychiatric diagnosis, treatment protocols, and patient management.

PSYCHIATRIC GUIDELINES:
1. Base responses on provided psychiatric literature
2. Provide evidence-based mental health information
3. Include diagnostic criteria (DSM-5), treatment protocols, and safety considerations
4. Use appropriate psychiatric terminology
5. Always consider safety and risk assessment
6. Emphasize the importance of professional psychiatric evaluation

MENTAL HEALTH KNOWLEDGE BASE:
{context}

CONVERSATION HISTORY:
{chat_history}

PSYCHIATRIC QUERY: {question}

CLINICAL RESPONSE:
Provide a comprehensive, evidence-based response addressing the mental health query. Include relevant diagnostic criteria, treatment options, safety considerations, and professional care recommendations.

Response:"""

        return PromptTemplate(
            template=template,
            input_variables=["context", "chat_history", "question"]
        )

class CardiacAgent(BaseMedicalAgent):
    """Specialized agent for cardiac/cardiovascular queries"""
    
    def __init__(self):
        super().__init__(MedicalDomain.CARDIAC, "Cardiac/Cardiovascular")
    
    def create_demo_knowledge_base(self) -> Tuple[int, int]:
        """Create demo cardiac knowledge base"""
        print("Creating demo cardiac knowledge base...")
        
        cardiac_content = [
            """
            Acute Myocardial Infarction (STEMI vs NSTEMI):
            
            STEMI (ST-Elevation MI):
            - ECG: ST elevation ≥2mm in precordial leads or ≥1mm in limb leads
            - Treatment: Primary PCI within 90 minutes or fibrinolytic therapy within 30 minutes
            - Antiplatelet therapy: Dual antiplatelet therapy (aspirin + P2Y12 inhibitor)
            - Anticoagulation: Heparin or bivalirudin
            
            NSTEMI (Non-ST-Elevation MI):
            - ECG: ST depression or T-wave inversion
            - Biomarkers: Elevated troponin
            - Risk stratification: TIMI or GRACE score
            - Treatment: Early invasive strategy vs. conservative management
            - Medical therapy: Beta-blockers, ACE inhibitors, statins
            """,
            """
            Heart Failure Classification and Management:
            
            NYHA Functional Classification:
            - Class I: No limitation of physical activity
            - Class II: Slight limitation of physical activity
            - Class III: Marked limitation of physical activity
            - Class IV: Unable to carry on physical activity without discomfort
            
            HFrEF (Heart Failure with Reduced Ejection Fraction):
            - EF ≤40%
            - Treatment: ACE inhibitors/ARBs, beta-blockers, aldosterone antagonists
            - Device therapy: ICD, CRT consideration
            
            HFpEF (Heart Failure with Preserved Ejection Fraction):
            - EF ≥50%
            - Treatment: Symptom management, treat comorbidities
            - Limited evidence for mortality benefit
            """,
            """
            Hypertension Guidelines and Management:
            
            2017 ACC/AHA Blood Pressure Categories:
            - Normal: <120/80 mmHg
            - Elevated: 120-129/<80 mmHg
            - Stage 1: 130-139/80-89 mmHg
            - Stage 2: ≥140/90 mmHg
            - Hypertensive Crisis: >180/120 mmHg
            
            First-line medications:
            - ACE inhibitors or ARBs
            - Thiazide or thiazide-like diuretics
            - Calcium channel blockers
            - Beta-blockers (specific indications)
            
            Target BP: <130/80 mmHg for most patients
            Lifestyle modifications: Diet, exercise, weight loss, sodium restriction
            """
        ]
        
        documents = []
        for i, content in enumerate(cardiac_content):
            doc = Document(
                page_content=content,
                metadata={
                    'source': f'cardiac_demo_section_{i+1}',
                    'topic': ['MI', 'Heart Failure', 'Hypertension'][i],
                    'domain': 'cardiac'
                }
            )
            documents.append(doc)
        
        self.knowledge_base = Chroma.from_documents(
            documents, 
            self.embeddings,
            collection_name="cardiac_demo"
        )
        
        print(f"Demo cardiac knowledge base created with {len(documents)} documents!")
        return len(documents), 1
    
    def create_medical_prompt(self) -> PromptTemplate:
        """Create cardiac-specific prompt template"""
        template = """You are CardiologyGPT, an advanced AI medical assistant specializing in cardiovascular medicine. You assist with cardiac diagnosis, treatment protocols, and cardiovascular patient management.

CARDIOLOGY GUIDELINES:
1. Base responses on provided cardiology literature
2. Provide evidence-based cardiovascular information
3. Include diagnostic criteria, treatment protocols, and risk stratification
4. Use appropriate cardiology terminology
5. Consider hemodynamic principles and cardiovascular physiology
6. Emphasize guideline-based care and risk reduction

CARDIOLOGY KNOWLEDGE BASE:
{context}

CONVERSATION HISTORY:
{chat_history}

CARDIOLOGY QUERY: {question}

CLINICAL RESPONSE:
Provide a comprehensive, evidence-based response addressing the cardiovascular query. Include relevant diagnostic criteria, treatment options, risk stratification, and cardiovascular risk reduction strategies.

Response:"""

        return PromptTemplate(
            template=template,
            input_variables=["context", "chat_history", "question"]
        )

class MedicalAgentRouter:
    """Router to determine which medical agent should handle the query"""
    
    def __init__(self):
        self.domain_keywords = {
            MedicalDomain.DIABETES: [
                'diabetes', 'diabetic', 'insulin', 'glucose', 'blood sugar', 'hba1c', 'metformin',
                'type 1 diabetes', 'type 2 diabetes', 'dka', 'diabetic ketoacidosis', 'hyperglycemia',
                'hypoglycemia', 'diabetic nephropathy', 'diabetic retinopathy', 'gestational diabetes'
            ],
            MedicalDomain.CANCER: [
                'cancer', 'oncology', 'tumor', 'malignancy', 'chemotherapy', 'radiation therapy',
                'metastasis', 'carcinoma', 'lymphoma', 'leukemia', 'staging', 'biopsy',
                'breast cancer', 'lung cancer', 'colon cancer', 'prostate cancer', 'melanoma'
            ],
            MedicalDomain.MENTAL_HEALTH: [
                'depression', 'anxiety', 'bipolar', 'schizophrenia', 'psychiatric', 'mental health',
                'antidepressant', 'mood disorder', 'panic', 'ptsd', 'adhd', 'autism',
                'psychotherapy', 'ssri', 'snri', 'antipsychotic', 'mood stabilizer'
            ],
            MedicalDomain.CARDIAC: [
                'heart', 'cardiac', 'cardiovascular', 'cardiology', 'myocardial infarction', 'mi',
                'heart failure', 'hypertension', 'arrhythmia', 'ecg', 'ekg', 'chest pain',
                'angina', 'coronary', 'valve', 'pericardial', 'aortic', 'mitral'
            ]
        }
    
    def route_query(self, query: str) -> MedicalDomain:
        """Route query to appropriate medical domain based on keywords"""
        query_lower = query.lower()
        
        # Count keyword matches for each domain
        domain_scores = {}
        for domain, keywords in self.domain_keywords.items():
            score = sum(1 for keyword in keywords if keyword in query_lower)
            if score > 0:
                domain_scores[domain] = score
        
        # Return domain with highest score, or GENERAL if no matches
        if domain_scores:
            return max(domain_scores, key=domain_scores.get)
        else:
            return MedicalDomain.GENERAL

class MedicalAssistantOrchestrator:
    """Main orchestrator that manages all medical agents and routing"""
    
    def __init__(self):
        print("🏥 Initializing Multi-Agent Medical Assistant System...")
        
        # Initialize router
        self.router = MedicalAgentRouter()
        
        # Initialize all medical agents
        self.agents = {
            MedicalDomain.DIABETES: DiabeticAgent(),
            MedicalDomain.CANCER: CancerAgent(),
            MedicalDomain.MENTAL_HEALTH: MentalHealthAgent(),
            MedicalDomain.CARDIAC: CardiacAgent()
        }
        
        # Shared conversation memory across all agents
        self.shared_memory = ConversationBufferWindowMemory(
            k=20,  # Remember more exchanges for multi-domain conversations
            memory_key="chat_history",
            return_messages=True,
            output_key="answer"
        )
        
        # Track which agent was last used
        self.last_agent_used = None
        self.conversation_count = 0
        
        print("✅ Multi-Agent Medical Assistant System initialized!")
    
    def load_knowledge_bases(self, pdf_configs: Dict[MedicalDomain, List[str]]):
        """Load knowledge bases for all agents"""
        print("\n" + "="*60)
        print("📚 LOADING MEDICAL KNOWLEDGE BASES")
        print("="*60)
        
        total_loaded = 0
        successful_agents = 0
        
        for domain, pdf_paths in pdf_configs.items():
            if domain in self.agents and pdf_paths:
                print(f"\n🔄 Loading {domain.value.title()} Knowledge Base...")
                try:
                    chunks, loads = self.agents[domain].load_knowledge_base(pdf_paths)
                    if chunks > 0:
                        self.agents[domain].setup_conversation_chain()
                        total_loaded += chunks
                        successful_agents += 1
                        print(f"✅ {domain.value.title()} agent ready with {chunks} knowledge chunks!")
                    else:
                        print(f"⚠️ {domain.value.title()} agent loaded with demo content")
                        self.agents[domain].setup_conversation_chain()
                        successful_agents += 1
                except Exception as e:
                    print(f"❌ Failed to setup {domain.value.title()} agent: {e}")
        
        print(f"\n📊 KNOWLEDGE BASE SUMMARY:")
        print(f"   - Total knowledge chunks loaded: {total_loaded}")
        print(f"   - Successful agents: {successful_agents}/{len(self.agents)}")
        print(f"   - System ready for multi-domain medical queries!")
        
        return total_loaded, successful_agents
    
    def chat(self, user_input: str) -> Dict:
        """Main chat interface that routes queries to appropriate agents"""
        self.conversation_count += 1
        
        # Route the query to determine which agent should handle it
        target_domain = self.router.route_query(user_input)
        
        # Handle general queries or fallback
        if target_domain == MedicalDomain.GENERAL:
            return self._handle_general_query(user_input)
        
        # Get the appropriate agent
        if target_domain not in self.agents:
            return self._handle_general_query(user_input)
        
        target_agent = self.agents[target_domain]
        
        # Check if agent is ready
        if target_agent.knowledge_base is None:
            return {
                "answer": f"I apologize, but the {target_agent.domain_name} specialist is not available right now. Please try a general medical question or contact support.",
                "source_documents": [],
                "confidence": "Low",
                "domain": target_domain.value,
                "agent": target_agent.domain_name,
                "routing_info": f"Routed to {target_agent.domain_name} but agent not ready"
            }
        
        # Get shared conversation history
        chat_history = self.shared_memory.chat_memory.messages
        
        # Ask the specialized agent
        try:
            response = target_agent.ask_question(user_input, chat_history)
            
            # Add agent switching notification if different from last agent
            if self.last_agent_used and self.last_agent_used != target_domain:
                agent_switch_note = f"\n\n🔄 *Switched to {target_agent.domain_name} specialist for this query*"
                response["answer"] += agent_switch_note
            
            # Update shared memory with the new exchange
            self.shared_memory.chat_memory.add_user_message(user_input)
            self.shared_memory.chat_memory.add_ai_message(response["answer"])
            
            # Track last agent used
            self.last_agent_used = target_domain
            
            # Add routing information
            response["routing_info"] = f"Routed to {target_agent.domain_name} specialist"
            response["conversation_count"] = self.conversation_count
            
            return response
            
        except Exception as e:
            error_response = {
                "answer": f"I encountered an error while consulting the {target_agent.domain_name} specialist: {str(e)}. Please try rephrasing your question.",
                "source_documents": [],
                "confidence": "Low",
                "domain": target_domain.value,
                "agent": target_agent.domain_name,
                "routing_info": f"Error in {target_agent.domain_name} agent",
                "conversation_count": self.conversation_count
            }
            
            # Still update memory to maintain conversation flow
            self.shared_memory.chat_memory.add_user_message(user_input)
            self.shared_memory.chat_memory.add_ai_message(error_response["answer"])
            
            return error_response
    
    def _handle_general_query(self, user_input: str) -> Dict:
        """Handle general medical queries that don't fit specific specialties"""
        
        general_response = f"""I'm a specialized medical assistant with expertise in diabetes, cancer/oncology, mental health/psychiatry, and cardiovascular medicine.

Your query: "{user_input}"

I couldn't identify a specific medical specialty for your question. Here are some options:

🩺 **Specialized Areas I Can Help With:**
- **Diabetes**: Blood sugar management, insulin therapy, diabetic complications
- **Cancer/Oncology**: Cancer staging, chemotherapy, treatment protocols
- **Mental Health**: Depression, anxiety, psychiatric medications
- **Cardiovascular**: Heart disease, hypertension, cardiac care

💡 **Try rephrasing your question with specific medical terms**, or ask me something like:
- "What are the symptoms of diabetes?"
- "How is breast cancer staged?"
- "What medications are used for depression?"
- "What causes heart attacks?"

I'm designed to provide evidence-based medical information to support clinical decision-making and medical education."""

        # Update shared memory
        self.shared_memory.chat_memory.add_user_message(user_input)
        self.shared_memory.chat_memory.add_ai_message(general_response)
        
        return {
            "answer": general_response,
            "source_documents": [],
            "confidence": "Medium",
            "domain": "general",
            "agent": "General Medical Assistant",
            "routing_info": "Handled as general query",
            "conversation_count": self.conversation_count
        }
    
    def get_system_status(self) -> Dict:
        """Get status of all agents and system"""
        status = {
            "total_agents": len(self.agents),
            "active_agents": 0,
            "agent_status": {},
            "conversation_count": self.conversation_count,
            "last_agent_used": self.last_agent_used.value if self.last_agent_used else None,
            "memory_messages": len(self.shared_memory.chat_memory.messages)
        }
        
        for domain, agent in self.agents.items():
            is_active = agent.knowledge_base is not None and agent.conversation_chain is not None
            if is_active:
                status["active_agents"] += 1
            
            status["agent_status"][domain.value] = {
                "name": agent.domain_name,
                "active": is_active,
                "has_knowledge_base": agent.knowledge_base is not None,
                "has_conversation_chain": agent.conversation_chain is not None
            }
        
        return status
    
    def clear_conversation_history(self):
        """Clear shared conversation history"""
        self.shared_memory.clear()
        self.last_agent_used = None
        self.conversation_count = 0
        print("🧹 Conversation history cleared across all agents.")
    
    def get_conversation_history(self):
        """Get current conversation history"""
        return self.shared_memory.chat_memory.messages

# Initialize the Multi-Agent Medical Assistant System
print("🚀 Starting Multi-Agent Medical Assistant System...")
medical_assistant = MedicalAssistantOrchestrator()

# =============================================================================
# PDF CONFIGURATION SECTION - MODIFY THESE PATHS FOR YOUR PDFs
# =============================================================================

# Configure PDF paths for each medical specialty
# Add your PDF file paths here - the system will handle missing files gracefully
PDF_CONFIGS = {
    MedicalDomain.DIABETES: [
        "diabetes_guidelines.pdf",
        "diabetes_textbook_1.pdf",
        "insulin_therapy.pdf"
    ],
    MedicalDomain.CANCER: [
        "chemotherapy_protocols.pdf",
        "oncology_textbook.pdf",
        "radiation_therapy.pdf"
    ],
    MedicalDomain.MENTAL_HEALTH: [
        "psychiatry_textbook.pdf",
        #"mental_health_handbook.pdf",
    ],
    MedicalDomain.CARDIAC: [
        "interventional_cardiology.pdf",
        "heart_failure_guidelines.pdf",
        "cardiology_textbook.pdf"
    ]
}


# Load knowledge bases for all agents
total_chunks, active_agents = medical_assistant.load_knowledge_bases(PDF_CONFIGS)

# =============================================================================
# INTERACTIVE CHAT INTERFACE
# =============================================================================

def start_medical_chat():
    """Start the interactive medical chat interface"""
    print("\n" + "="*80)
    print("🏥 MULTI-AGENT MEDICAL ASSISTANT - CHAT INTERFACE")
    print("="*80)
    print("💡 I'm a specialized medical AI with expertise in:")
    print("   • 🩺 Diabetes & Endocrinology")
    print("   • 🎗️ Cancer & Oncology") 
    print("   • 🧠 Mental Health & Psychiatry")
    print("   • ❤️ Cardiovascular Medicine")
    print("\n📝 Commands:")
    print("   • Type 'status' to see system status")
    print("   • Type 'clear' to clear conversation history")
    print("   • Type 'quit' or 'exit' to end chat")
    print("   • Type 'demo' to see example queries")
    print("\n🚀 Ready for your medical questions!")
    print("-" * 80)
    
    while True:
        try:
            # Get user input
            user_input = input("\n👩‍⚕️ You: ").strip()
            
            # Handle special commands
            if user_input.lower() in ['quit', 'exit', 'q']:
                print("\n👋 Thank you for using the Multi-Agent Medical Assistant!")
                break
            
            elif user_input.lower() == 'clear':
                medical_assistant.clear_conversation_history()
                continue
            
            elif user_input.lower() == 'status':
                status = medical_assistant.get_system_status()
                print(f"\n📊 SYSTEM STATUS:")
                print(f"   • Active Agents: {status['active_agents']}/{status['total_agents']}")
                print(f"   • Total Conversations: {status['conversation_count']}")
                print(f"   • Last Agent Used: {status['last_agent_used']}")
                print(f"   • Memory Messages: {status['memory_messages']}")
                print(f"\n🤖 Agent Status:")
                for domain, info in status['agent_status'].items():
                    status_icon = "✅" if info['active'] else "❌"
                    print(f"   {status_icon} {info['name']}: {'Active' if info['active'] else 'Inactive'}")
                continue
            
            elif user_input.lower() == 'demo':
                show_demo_queries()
                continue
            
            elif not user_input:
                print("Please enter a medical question or command.")
                continue
            
            # Process medical query
            print(f"\n🤔 Processing your medical query...")
            response = medical_assistant.chat(user_input)
            
            # Display response
            print(f"\n🩺 {response['agent']} ({response['confidence']} Confidence):")
            print("-" * 60)
            print(response['answer'])
            
            # Show source information if available
            if response['source_documents']:
                print(f"\n📚 Sources: {len(response['source_documents'])} relevant documents found")
            
            # Show routing information
            print(f"\n🔄 {response['routing_info']}")
            
        except KeyboardInterrupt:
            print("\n\n👋 Chat interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")
            print("Please try again with a different question.")

def show_demo_queries():
    """Show example queries for each medical specialty"""
    print("\n" + "="*60)
    print("💡 EXAMPLE MEDICAL QUERIES")
    print("="*60)
    
    examples = {
        "🩺 Diabetes": [
            "What are the diagnostic criteria for Type 2 diabetes?",
            "How do we manage diabetic ketoacidosis?",
            "What are the contraindications for metformin?"
        ],
        "🎗️ Cancer/Oncology": [
            "How is breast cancer staged?",
            "What are the side effects of chemotherapy?",
            "What's the difference between NSCLC and SCLC?"
        ],
        "🧠 Mental Health": [
            "What are the DSM-5 criteria for major depression?",
            "How do we treat generalized anxiety disorder?",
            "What medications are used for bipolar disorder?"
        ],
        "❤️ Cardiovascular": [
            "What's the difference between STEMI and NSTEMI?",
            "How do we classify heart failure?",
            "What are the current hypertension guidelines?"
        ]
    }
    
    for specialty, queries in examples.items():
        print(f"\n{specialty}:")
        for query in queries:
            print(f"   • {query}")
    
    print(f"\n💬 Try asking any of these questions to see the multi-agent system in action!")

def demonstrate_multi_agent_system():
    """Demonstrate the multi-agent system with example queries"""
    print("\n" + "="*80)
    print("🎯 MULTI-AGENT MEDICAL ASSISTANT DEMONSTRATION")
    print("="*80)
    
    # Example queries that will trigger different agents
    demo_queries = [
        ("What are the diagnostic criteria for Type 2 diabetes?", MedicalDomain.DIABETES),
        ("How is breast cancer staged and treated?", MedicalDomain.CANCER),
        ("What are the DSM-5 criteria for major depression?", MedicalDomain.MENTAL_HEALTH),
        ("What's the difference between STEMI and NSTEMI?", MedicalDomain.CARDIAC),
        ("Can diabetes increase the risk of heart disease?", MedicalDomain.DIABETES),  # Should route to diabetes
        ("Tell me about anxiety disorders", MedicalDomain.MENTAL_HEALTH)
    ]
    
    for i, (query, expected_domain) in enumerate(demo_queries, 1):
        print(f"\n" + "="*60)
        print(f"🔍 Demo Query {i}: {query}")
        print("-" * 60)
        
        try:
            response = medical_assistant.chat(query)
            
            print(f"🤖 Agent Used: {response['agent']}")
            print(f"🎯 Domain: {response['domain']}")
            print(f"📊 Confidence: {response['confidence']}")
            print(f"🔄 Routing: {response['routing_info']}")
            print(f"\n💬 Response Preview:")
            print(response['answer'][:300] + "..." if len(response['answer']) > 300 else response['answer'])
            
            if response['source_documents']:
                print(f"\n📚 {len(response['source_documents'])} knowledge sources referenced")
                
        except Exception as e:
            print(f"❌ Error in demo query {i}: {e}")
        
        print("-" * 60)
    
    # Show system status after demo
    print(f"\n📊 Final System Status:")
    status = medical_assistant.get_system_status()
    print(f"   • Total Conversations: {status['conversation_count']}")
    print(f"   • Active Agents: {status['active_agents']}/{status['total_agents']}")
    print(f"   • Memory Messages: {status['memory_messages']}")

# Run system demonstration if agents are loaded
if active_agents > 0:
    print(f"\n✅ System ready with {active_agents} active medical agents!")
    
    # Show setup instructions
    print(f"""
📁 SETUP INSTRUCTIONS FOR YOUR PDFs:
===================================

To add your own medical textbooks:

1. **Update PDF paths** in the PDF_CONFIGS section above
2. **Supported locations**:
   • Same directory: "textbook.pdf"
   • Full path: "/path/to/your/textbook.pdf"
   • Multiple files per specialty

3. **Current configuration**:""")
    
    for domain, paths in PDF_CONFIGS.items():
        print(f"   • {domain.value.title()}: {len(paths)} files configured")
        for path in paths[:2]:  # Show first 2 paths
            exists = "✅" if os.path.exists(path) else "❌"
            print(f"     {exists} {path}")
        if len(paths) > 2:
            print(f"     ... and {len(paths)-2} more files")
    
    print(f"\n🚀 READY TO USE! Choose an option:")
    print(f"   1. Type: start_medical_chat() - for interactive chat")
    print(f"   2. Type: demonstrate_multi_agent_system() - for automated demo")
    print(f"   3. Use directly: medical_assistant.chat('your question')")
    
else:
    print(f"\n⚠️ No agents successfully loaded. Using demo mode.")
    print(f"Please check your PDF paths in the PDF_CONFIGS section.")

# Auto-start demo if no PDFs are found
if total_chunks == 0:
    print(f"\n🎮 Starting automated demonstration with demo content...")
    demonstrate_multi_agent_system()

print(f"\n" + "="*80)
print(f"🏥 MULTI-AGENT MEDICAL ASSISTANT SYSTEM READY")
print(f"="*80)
print(f"💡 Usage Examples:")
print(f"   • medical_assistant.chat('What is diabetes?')")
print(f"   • start_medical_chat()  # Interactive mode")
print(f"   • medical_assistant.get_system_status()")
print(f"   • medical_assistant.clear_conversation_history()")


🚀 Starting Multi-Agent Medical Assistant System...
🏥 Initializing Multi-Agent Medical Assistant System...
✅ Multi-Agent Medical Assistant System initialized!

📚 LOADING MEDICAL KNOWLEDGE BASES

🔄 Loading Diabetes Knowledge Base...
🔄 Loading Diabetes textbook 1: diabetes_guidelines.pdf
Loaded 132 pages from Diabetes textbook 1
✅ Successfully added Diabetes textbook 1
🔄 Loading Diabetes textbook 2: diabetes_textbook_1.pdf
Loaded 1141 pages from Diabetes textbook 2
✅ Successfully added Diabetes textbook 2
🔄 Loading Diabetes textbook 3: insulin_therapy.pdf
Loaded 138 pages from Diabetes textbook 3
✅ Successfully added Diabetes textbook 3
📖 Total Diabetes documents: 1411 pages
Created 8767 text chunks for Diabetes


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Embedding error for Diabetes: 'results'
Trying Diabetes knowledge base creation with smaller batches...
Processed Diabetes batch 1/176
Processed Diabetes batch 2/176
Processed Diabetes batch 3/176
Processed Diabetes batch 4/176
Processed Diabetes batch 5/176
Processed Diabetes batch 6/176
Processed Diabetes batch 7/176
Processed Diabetes batch 8/176
Processed Diabetes batch 9/176
Processed Diabetes batch 10/176
Processed Diabetes batch 11/176
Processed Diabetes batch 12/176
Processed Diabetes batch 13/176
Processed Diabetes batch 14/176
Processed Diabetes batch 15/176
Processed Diabetes batch 16/176
Processed Diabetes batch 17/176
Processed Diabetes batch 18/176
Processed Diabetes batch 19/176
Processed Diabetes batch 20/176
Processed Diabetes batch 21/176
Processed Diabetes batch 22/176
Processed Diabetes batch 23/176
Processed Diabetes batch 24/176
Processed Diabetes batch 25/176
Processed Diabetes batch 26/176
Processed Diabetes batch 27/176
Processed Diabetes batch 28/176
Processed

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Loaded 373 pages from Cancer/Oncology textbook 3
✅ Successfully added Cancer/Oncology textbook 3
📖 Total Cancer/Oncology documents: 1746 pages
Created 6993 text chunks for Cancer/Oncology


...

Embedding error for Mental Health/Psychiatry: 'results'
Trying Mental Health/Psychiatry knowledge base creation with smaller batches...
Processed Mental Health/Psychiatry batch 1/23
Processed Mental Health/Psychiatry batch 2/23
Processed Mental Health/Psychiatry batch 3/23
Processed Mental Health/Psychiatry batch 4/23
Processed Mental Health/Psychiatry batch 5/23
Processed Mental Health/Psychiatry batch 6/23
Processed Mental Health/Psychiatry batch 7/23
Processed Mental Health/Psychiatry batch 8/23
Processed Mental Health/Psychiatry batch 9/23
Processed Mental Health/Psychiatry batch 10/23
Processed Mental Health/Psychiatry batch 11/23
Processed Mental Health/Psychiatry batch 12/23
Processed Mental Health/Psychiatry batch 13/23
Processed Mental Health/Psychiatry batch 14/23
Processed Mental Health/Psychiatry batch 15/23
Processed Mental Health/Psychiatry batch 16/23
Processed Mental Health/Psychiatry batch 17/23
Processed Mental Health/Psychiatry batch 18/23
Processed Mental Health/Psy

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Loaded 38 pages from Cardiac/Cardiovascular textbook 3
✅ Successfully added Cardiac/Cardiovascular textbook 3
📖 Total Cardiac/Cardiovascular documents: 941 pages
Created 6255 text chunks for Cardiac/Cardiovascular


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Embedding error for Cardiac/Cardiovascular: 'results'
Trying Cardiac/Cardiovascular knowledge base creation with smaller batches...
Processed Cardiac/Cardiovascular batch 1/126
Processed Cardiac/Cardiovascular batch 2/126
Processed Cardiac/Cardiovascular batch 3/126
Processed Cardiac/Cardiovascular batch 4/126
Processed Cardiac/Cardiovascular batch 5/126
Processed Cardiac/Cardiovascular batch 6/126
Processed Cardiac/Cardiovascular batch 7/126
Processed Cardiac/Cardiovascular batch 8/126
Processed Cardiac/Cardiovascular batch 9/126
Processed Cardiac/Cardiovascular batch 10/126
Processed Cardiac/Cardiovascular batch 11/126
Processed Cardiac/Cardiovascular batch 12/126
Processed Cardiac/Cardiovascular batch 13/126
Processed Cardiac/Cardiovascular batch 14/126
Processed Cardiac/Cardiovascular batch 15/126
Processed Cardiac/Cardiovascular batch 16/126
Processed Cardiac/Cardiovascular batch 17/126
Processed Cardiac/Cardiovascular batch 18/126
Processed Cardiac/Cardiovascular batch 19/126
Pro

In [2]:
start_medical_chat()  # Start interactive chat



🏥 MULTI-AGENT MEDICAL ASSISTANT - CHAT INTERFACE
💡 I'm a specialized medical AI with expertise in:
   • 🩺 Diabetes & Endocrinology
   • 🎗️ Cancer & Oncology
   • 🧠 Mental Health & Psychiatry
   • ❤️ Cardiovascular Medicine

📝 Commands:
   • Type 'status' to see system status
   • Type 'clear' to clear conversation history
   • Type 'quit' or 'exit' to end chat
   • Type 'demo' to see example queries

🚀 Ready for your medical questions!
--------------------------------------------------------------------------------



👩‍⚕️ You:  hello



🤔 Processing your medical query...

🩺 General Medical Assistant (Medium Confidence):
------------------------------------------------------------
I'm a specialized medical assistant with expertise in diabetes, cancer/oncology, mental health/psychiatry, and cardiovascular medicine.

Your query: "hello"

I couldn't identify a specific medical specialty for your question. Here are some options:

🩺 **Specialized Areas I Can Help With:**
- **Diabetes**: Blood sugar management, insulin therapy, diabetic complications
- **Cancer/Oncology**: Cancer staging, chemotherapy, treatment protocols
- **Mental Health**: Depression, anxiety, psychiatric medications
- **Cardiovascular**: Heart disease, hypertension, cardiac care

💡 **Try rephrasing your question with specific medical terms**, or ask me something like:
- "What are the symptoms of diabetes?"
- "How is breast cancer staged?"
- "What medications are used for depression?"
- "What causes heart attacks?"

I'm designed to provide evidence-based


👩‍⚕️ You:  what is a disease ?



🤔 Processing your medical query...

🩺 General Medical Assistant (Medium Confidence):
------------------------------------------------------------
I'm a specialized medical assistant with expertise in diabetes, cancer/oncology, mental health/psychiatry, and cardiovascular medicine.

Your query: "what is a disease ?"

I couldn't identify a specific medical specialty for your question. Here are some options:

🩺 **Specialized Areas I Can Help With:**
- **Diabetes**: Blood sugar management, insulin therapy, diabetic complications
- **Cancer/Oncology**: Cancer staging, chemotherapy, treatment protocols
- **Mental Health**: Depression, anxiety, psychiatric medications
- **Cardiovascular**: Heart disease, hypertension, cardiac care

💡 **Try rephrasing your question with specific medical terms**, or ask me something like:
- "What are the symptoms of diabetes?"
- "How is breast cancer staged?"
- "What medications are used for depression?"
- "What causes heart attacks?"

I'm designed to provide 

...


🤔 Processing your medical query...


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given



🩺 Cancer/Oncology (High Confidence):
------------------------------------------------------------


Cancer can affect virtually any organ or tissue in the body. The type of cancer and its stage determine the extent of involvement and the appropriate treatment strategies. Below is an overview of some common types of cancer and their potential impacts on different organs:

### Common Types of Cancer and Their Effects:

1. **Lung Cancer:**
   - **Organs Affected:** Primarily affects the lungs but can metastasize to the brain, bones, liver, and adrenal glands.
   - **Staging:** TNM system (Tumor, Node, Metastasis) is used for staging.
   - **Treatment Options:** Surgery, chemotherapy, radiation therapy, targeted therapies, and immunotherapy.
   - **Side Effect Management:** Manage respiratory symptoms, pain, fatigue, and side effects of treatments.

2. **Breast Cancer:**
   - **Organs Affected:** Primarily affects the breasts but can spread to lymph nodes, bones, lungs, liver, and brain.



👩‍⚕️ You:  quit



👋 Thank you for using the Multi-Agent Medical Assistant!


In [3]:
status = medical_assistant.get_system_status()
print(status)

{'total_agents': 4, 'active_agents': 4, 'agent_status': {'diabetes': {'name': 'Diabetes', 'active': True, 'has_knowledge_base': True, 'has_conversation_chain': True}, 'cancer': {'name': 'Cancer/Oncology', 'active': True, 'has_knowledge_base': True, 'has_conversation_chain': True}, 'mental_health': {'name': 'Mental Health/Psychiatry', 'active': True, 'has_knowledge_base': True, 'has_conversation_chain': True}, 'cardiac': {'name': 'Cardiac/Cardiovascular', 'active': True, 'has_knowledge_base': True, 'has_conversation_chain': True}}, 'conversation_count': 6, 'last_agent_used': 'cancer', 'memory_messages': 20}


In [4]:
history = medical_assistant.get_conversation_history()
print(history)

[HumanMessage(content='hello'), AIMessage(content='I\'m a specialized medical assistant with expertise in diabetes, cancer/oncology, mental health/psychiatry, and cardiovascular medicine.\n\nYour query: "hello"\n\nI couldn\'t identify a specific medical specialty for your question. Here are some options:\n\n🩺 **Specialized Areas I Can Help With:**\n- **Diabetes**: Blood sugar management, insulin therapy, diabetic complications\n- **Cancer/Oncology**: Cancer staging, chemotherapy, treatment protocols\n- **Mental Health**: Depression, anxiety, psychiatric medications\n- **Cardiovascular**: Heart disease, hypertension, cardiac care\n\n💡 **Try rephrasing your question with specific medical terms**, or ask me something like:\n- "What are the symptoms of diabetes?"\n- "How is breast cancer staged?"\n- "What medications are used for depression?"\n- "What causes heart attacks?"\n\nI\'m designed to provide evidence-based medical information to support clinical decision-making and medical educat

In [5]:
response = medical_assistant.chat("What are the symptoms of diabetes?")
print(response["answer"])




---

**Symptoms of Diabetes:**

Diabetes mellitus, whether Type 1 or Type 2, can present with a range of symptoms. Early recognition of these symptoms is crucial for timely diagnosis and management.

### Common Symptoms:

1. **Polyuria (Frequent Urination):**
   - Excessive urine output due to the kidneys attempting to eliminate excess glucose.

2. **Polydipsia (Excessive Thirst):**
   - Increased thirst resulting from fluid loss through frequent urination.

3. **Polyphagia (Increased Hunger):**
   - Despite eating, the body may not be able to utilize glucose effectively, leading to persistent hunger.

4. **Fatigue:**
   - Generalized weakness and tiredness due to the body's inability to efficiently convert glucose into energy.

5. **Blurred Vision:**
   - High blood sugar levels can cause temporary changes in the shape of the lens, affecting vision.

6. **Weight Loss:**
   - Unexplained weight loss despite increased appetite, particularly in Type 1 diabetes.

7. **Slow Healing Wound

In [6]:
response = medical_assistant.chat("What are the early symptoms of Type 2 diabetes and how are they different from Type 1?")
print(response["answer"])

response = medical_assistant.chat("What is the significance of HbA1c in diabetes diagnosis, and what value indicates prediabetes?")
print(response["answer"])

response = medical_assistant.chat("How does Metformin work in managing blood sugar levels in diabetic patients?")
print(response["answer"])

response = medical_assistant.chat("Why might a patient with Type 1 diabetes require both basal and bolus insulin?")
print(response["answer"])

response = medical_assistant.chat("Can a person with Type 2 diabetes follow a ketogenic diet safely? What are the pros and cons?")
print(response["answer"])

response = medical_assistant.chat("How does physical activity influence insulin sensitivity in diabetic patients?")
print(response["answer"])

response = medical_assistant.chat("What is diabetic neuropathy, and how is it managed?")
print(response["answer"])

response = medical_assistant.chat("What should be done immediately if a diabetic person is found unconscious and hypoglycemic?")
print(response["answer"])

response = medical_assistant.chat("What role does GLP-1 play in diabetes treatment, and how do GLP-1 receptor agonists work?")
print(response["answer"])

response = medical_assistant.chat("If I’m 45, overweight, and have a family history of diabetes, should I get tested even if I have no symptoms? Why?")
print(response["answer"])




---

**Early Symptoms of Type 2 Diabetes and Comparison with Type 1 Diabetes:**

### Early Symptoms of Type 2 Diabetes:

1. **Gradual Onset:**
   - Unlike Type 1 diabetes, Type 2 diabetes typically develops gradually over time. Symptoms may be mild and go unnoticed for years.

2. **Polyuria (Frequent Urination):**
   - Increased frequency of urination, especially at night.

3. **Polydipsia (Excessive Thirst):**
   - Persistent thirst due to dehydration caused by frequent urination.

4. **Fatigue:**
   - Feeling unusually tired or weak, even with adequate rest.

5. **Blurred Vision:**
   - Changes in vision due to high blood sugar levels affecting the eyes.

6. **Slow Healing Wounds:**
   - Delayed healing of cuts, bruises, or sores, particularly on the feet.

7. **Recurrent Infections:**
   - Increased susceptibility to infections, such as yeast infections, urinary tract infections, and skin infections.

8. **Numbness or Tingling in Extremities:**
   - Peripheral neuropathy causing n

Failure during generate. (POST https://us-south.ml.cloud.ibm.com/ml/v1/text/embeddings?version=2024-05-10)
Status code: 429, body: {"error":"You have exceeded your usage quota for Skills Network-provided us-south.ml.cloud.ibm.com for the day. Your quota will reset in 19 hours 22 minutes from now."}


I apologize, but I encountered an error processing your Diabetes query: Failure during generate. (POST https://us-south.ml.cloud.ibm.com/ml/v1/text/embeddings?version=2024-05-10)
Status code: 429, body: {"error":"You have exceeded your usage quota for Skills Network-provided us-south.ml.cloud.ibm.com for the day. Your quota will reset in 19 hours 22 minutes from now."}. Please try rephrasing your question.


Failure during generate. (POST https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-05-10)
Status code: 429, body: {"error":"You have exceeded your usage quota for Skills Network-provided us-south.ml.cloud.ibm.com for the day. Your quota will reset in 19 hours 22 minutes from now."}


I apologize, but I encountered an error processing your Diabetes query: Failure during generate. (POST https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-05-10)
Status code: 429, body: {"error":"You have exceeded your usage quota for Skills Network-provided us-south.ml.cloud.ibm.com for the day. Your quota will reset in 19 hours 22 minutes from now."}. Please try rephrasing your question.


...

I apologize, but I encountered an error processing your Diabetes query: Failure during generate. (POST https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-05-10)
Status code: 429, body: {"error":"You have exceeded your usage quota for Skills Network-provided us-south.ml.cloud.ibm.com for the day. Your quota will reset in 19 hours 22 minutes from now."}. Please try rephrasing your question.


Failure during generate. (POST https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-05-10)
Status code: 429, body: {"error":"You have exceeded your usage quota for Skills Network-provided us-south.ml.cloud.ibm.com for the day. Your quota will reset in 19 hours 22 minutes from now."}


I apologize, but I encountered an error processing your Diabetes query: Failure during generate. (POST https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-05-10)
Status code: 429, body: {"error":"You have exceeded your usage quota for Skills Network-provided us-south.ml.cloud.ibm.com for the day. Your quota will reset in 19 hours 22 minutes from now."}. Please try rephrasing your question.


Failure during generate. (POST https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-05-10)
Status code: 429, body: {"error":"You have exceeded your usage quota for Skills Network-provided us-south.ml.cloud.ibm.com for the day. Your quota will reset in 19 hours 22 minutes from now."}


I apologize, but I encountered an error processing your Diabetes query: Failure during generate. (POST https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-05-10)
Status code: 429, body: {"error":"You have exceeded your usage quota for Skills Network-provided us-south.ml.cloud.ibm.com for the day. Your quota will reset in 19 hours 22 minutes from now."}. Please try rephrasing your question.


In [ ]:
start_medical_chat()  # Start interactive chat


🏥 MULTI-AGENT MEDICAL ASSISTANT - CHAT INTERFACE
💡 I'm a specialized medical AI with expertise in:
   • 🩺 Diabetes & Endocrinology
   • 🎗️ Cancer & Oncology
   • 🧠 Mental Health & Psychiatry
   • ❤️ Cardiovascular Medicine

📝 Commands:
   • Type 'status' to see system status
   • Type 'clear' to clear conversation history
   • Type 'quit' or 'exit' to end chat
   • Type 'demo' to see example queries

🚀 Ready for your medical questions!
--------------------------------------------------------------------------------



👩‍⚕️ You:  what is your name



🤔 Processing your medical query...

🩺 General Medical Assistant (Medium Confidence):
------------------------------------------------------------
I'm a specialized medical assistant with expertise in diabetes, cancer/oncology, mental health/psychiatry, and cardiovascular medicine.

Your query: "what is your name"

I couldn't identify a specific medical specialty for your question. Here are some options:

🩺 **Specialized Areas I Can Help With:**
- **Diabetes**: Blood sugar management, insulin therapy, diabetic complications
- **Cancer/Oncology**: Cancer staging, chemotherapy, treatment protocols
- **Mental Health**: Depression, anxiety, psychiatric medications
- **Cardiovascular**: Heart disease, hypertension, cardiac care

💡 **Try rephrasing your question with specific medical terms**, or ask me something like:
- "What are the symptoms of diabetes?"
- "How is breast cancer staged?"
- "What medications are used for depression?"
- "What causes heart attacks?"

I'm designed to provide ev


👩‍⚕️ You:  what is diabetes



🤔 Processing your medical query...

🩺 Diabetes (High Confidence):
------------------------------------------------------------


Diabetes mellitus is a group of metabolic disorders characterized by chronic hyperglycemia resulting from defects in insulin secretion, insulin action, or both. The three main types of diabetes are:

1. Type 1 Diabetes (T1D): An autoimmune disorder causing destruction of pancreatic beta cells, leading to absolute insulin deficiency.

2. Type 2 Diabetes (T2D): A progressive metabolic disorder characterized by insulin resistance and relative insulin deficiency.

3. Gestational Diabetes Mellitus (GDM): Hyperglycemia first detected during pregnancy, often resolving after delivery.

Pathophysiology:

Insulin is a key hormone regulating glucose homeostasis. Insulin resistance refers to reduced insulin sensitivity in target tissues, requiring higher insulin concentrations to achieve normal glucose uptake and utilization. Beta cell dysfunction results in decreased i


👩‍⚕️ You:  what is cancer



🤔 Processing your medical query...

🩺 Cancer/Oncology (High Confidence):
------------------------------------------------------------


Cancer is a complex group of diseases characterized by uncontrolled cell growth and potential spread throughout the body. There are over 100 distinct types of cancer, categorized based on the originating tissue or organ. Key examples include carcinomas (epithelial tissue), sarcomas (connective tissue), leukemias (bone marrow), lymphomas (immune system), and brain tumors.

Carcinogenesis, the process of cancer development, unfolds in three primary stages: initiation, promotion, and progression. Initiation involves genetic mutations triggered by environmental carcinogens (e.g., tobacco smoke, UV radiation, chemicals) or inherited genetic susceptibility. During promotion, cells with initiated mutations multiply and accrue additional genetic alterations due to sustained exposure to promoters (e.g., hormones, inflammation). Finally, progression entails fur